In [ ]:
! pip install -qU sagemaker
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket()  
print("S3 bucket name: ", bucket)

In [ ]:
import os
os.environ["S3_BUCKET"] = bucket

!echo ${S3_BUCKET}

Upload to S3

In [ ]:
!aws s3 cp . s3://${S3_BUCKET}/data-preparation-using-amazon-sagemaker-and-glue-databrew --recursive

Run the next code cell to get the pre-built Docker Image URL for scikit-learn container, stored in Amazon ECR.

In [ ]:
import boto3

region = boto3.session.Session().region_name
accountRegionMap =	{
  "us-west-1": "746614075791","us-west-2": "246618743249","us-east-1": "683313688378",
  "us-east-2": "257758044811","ap-northeast-1": "354813040037","ap-northeast-2": "354813040037",
  "ap-southeast-1": "121021644041","ap-southeast-2": "783357654285","ap-south-1": "720646828776",
  "eu-west-1": "141502667606","eu-west-2": "764974769150","eu-central-1": "492215442770","ca-central-1": "341280168497"
}

account_id = accountRegionMap[region]
ecr_repository = "sagemaker-scikit-learn"
tag = ":0.23-1-cpu-py3"

uri_suffix = "amazonaws.com"
if region in ["cn-north-1", "cn-northwest-1"]:
    uri_suffix = "amazonaws.com.cn"
ecr_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
    account_id, region, uri_suffix, ecr_repository + tag
)

print("SageMaker processing repository uri: ", ecr_repository_uri)


On the Data Processing Step set the source and destination S3 URL for processing job.

In [ ]:
source = 's3://'+bucket+'/data-preparation-using-amazon-sagemaker-and-glue-databrew/DataSet/insurance_claims.csv'
destination = 's3://'+bucket+'/data-preparation-using-amazon-sagemaker-and-glue-databrew/Results/DataProcessing'

Run the next cell to generate the AutoInsuranceFraudProcessing.py file. All the processing steps are defined here.

In [ ]:
%%writefile AutoInsuranceFraudProcessing.py
#This block of code generates a file "AutoInsuranceFraudProcessing.py" which has the code to process the data

import argparse
import os
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings(action="ignore", category=DataConversionWarning)

if __name__ == "__main__":
    #get arguments
    parser = argparse.ArgumentParser()
    args, _ = parser.parse_known_args()
    print("Received arguments {}".format(args))
    
    #get the input data
    input_data_path = os.path.join("/opt/ml/processing/input", "insurance_claims.csv")
    print("Reading input data from {}".format(input_data_path))
    df = pd.read_csv(input_data_path)
    df = pd.DataFrame(data=df)
    print(df.head())

    #replacing ? with nan for the columns
    df['police_report_available']=df['police_report_available'].replace('?',np.nan)
    #dropping the unnecessary rows
    df=df.dropna(subset=['police_report_available'])
    
    #drop unnecessary columns
    df=df.drop(['months_as_customer'],axis=1)
    
    #now deal with the categorical features
    #for the columns insured_sex and fraud_reported
    le=LabelEncoder()
    for i in list(df.columns):
        if df[i].dtype=='object':
            df[i]=le.fit_transform(df[i])
    
    #final preprocessed data
    print(df.head())
    train_features_output_path = os.path.join("/opt/ml/processing/output", "preprocessed_data.csv")
    df.to_csv(train_features_output_path, index=False)
    print("done")
    

In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=ecr_repository_uri,
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.m5.2xlarge')

script_processor.run(code='AutoInsuranceFraudProcessing.py',
                     inputs=[ProcessingInput(source=source,
                                             destination='/opt/ml/processing/input')],
                     outputs=[ProcessingOutput(output_name="preprocessed_data.csv", destination=destination,
                                               source='/opt/ml/processing/output')])

You can view the job by going into the SageMaker console > Processing > Processing jobs

Run the last cell to export the data to S3

In [ ]:
preprocessing_job_description = script_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    print(output)
    preprocessed_data = output["S3Output"]["S3Uri"]
    print(preprocessed_data)